In [1]:
import sys
sys.path.insert(0, '..')

import yaml
import numpy as np
import jax.numpy as jnp
import pandas as pd
from jax import vmap

# Creating a combined config file:

Loading files:

In [2]:
# Position of hexagon pixels on sensor:
df = pd.read_csv('pixel_pos.csv', names=['x', 'y'])
pix_y = df[['y']].values.flatten()
pix_x = df[['x']].values.flatten()

In [3]:
# Positions of mirrors:
df = pd.read_csv('hess_mirrors.dat', comment='#', sep='\s+')
df = df.apply(lambda x: x.astype(str).str.replace(',', ''))
df = df.astype(float)

mirrors_xyd = np.array(df[['y', 'x', 'p']].values)/100

In [4]:
# Positions of shadowing structure:
df = pd.read_csv('hess_masts.dat', comment='#', sep='\s+')
df = df.apply(lambda x: x.astype(str).str.replace(',', ''))
df = df.astype(float)

cyl_points1 = np.array(df[['x1', 'y1', 'z1']].values)*0.01
cyl_points2 = np.array(df[['x2', 'y2', 'z2']].values)*0.01
cyl_radius = np.array(df[['d']].values/2)*0.01

# Some useful code:

In [5]:
def spherical_surface(radius, points):
    Z = radius-np.sqrt(radius**2-(points[...,0]**2+points[...,1]**2))
    N = np.stack([-points[...,0], -points[...,1], radius - Z], axis=-1)
    return np.stack([points[...,0], points[...,1], Z], axis=-1), N/radius

def look_at_euler(mirror_pos, target_pos=jnp.array([0., 0., 0.]), up=jnp.array([0., 1., 0.])):   
    # Build rotation matrix
    forward = target_pos - mirror_pos
    forward = forward / jnp.linalg.norm(forward)
    right = jnp.cross(forward, up)
    right = right / jnp.linalg.norm(right)
    up_corrected = jnp.cross(right, forward)
    
    R = jnp.column_stack([right, up_corrected, forward])
    
    # Extract tilt (rotation around Y)
    tilt = jnp.arcsin(R[0, 2])
    
    # Extract tip (rotation around X)
    tip = jnp.arctan2(-R[1, 2], R[2, 2])
    
    # Extract rotation (rotation around Z)
    rotation = jnp.arctan2(-R[0, 1], R[0, 0])
    
    # Convert to degrees
    return jnp.array([
        jnp.rad2deg(tip),
        jnp.rad2deg(tilt),
        jnp.rad2deg(rotation)
    ])

# Using ConfigBuilder:

In [6]:
from builder import TelescopeConfigBuilder

In [7]:
builder = TelescopeConfigBuilder('CT1')

# Hexagonal sensor
builder.add_hexagon_sensor_array(
    sensor_id='science_camera',
    position=[0, 0, 15.028],
    orientation=[0, 180, 0],
    pixel_x = pix_x,
    pixel_y = pix_y)

# Square lid sensor
builder.add_square_sensor_array(
    sensor_id='camera_lid',
    position=[0, 0, 15.0],
    orientation=[0, 180, 0],
    width = 1000,
    height = 1000,
    bounds = [-0.8,0.8,-0.8,0.8])

# Mirror array
## Add spherical template:
builder.add_mirror_template('spherical_30m', curvature=1/30, conic=0.0, aspheric_coeffs=[])

## For all mirrors, calculate position and rotation:
Tmirrors, _ = spherical_surface(15, mirrors_xyd[:,:2])
Rmirrors = np.array(vmap(look_at_euler, in_axes= (0, None))(Tmirrors, jnp.array([0,0,30])))
Rmirrors[:,2] = 0
## Add mirrors one by one
for i in range(len(Tmirrors)):
    builder.add_mirror_circular(
        mirror_id='M_{}'.format(i),
        template='spherical_30m',
        position=Tmirrors[i],
        orientation=Rmirrors[i],
        radius=0.3
        )
    
# Add shadowing obstructions:
for i in range(len(cyl_points1)):
    if i<4:
        cyl_string = 'Main_Mast_{}'.format(i)
    if i>=4 and i < 8:
        cyl_string = 'Interconnecting_Mast_{}'.format(i-4)
    if i>=8 and i < 24:
        cyl_string = 'Stabilizing_String_{}'.format(i-8)
    if i>=24 and i < 32:
        cyl_string = 'Camera_Mounting_Frame_{}'.format(i-24)
    if i==32:
        cyl_string = 'Camera_Body'
        
    builder.add_obstruction_cylinder(cyl_string,
                                     cyl_points1[i],
                                     cyl_points2[i],
                                     cyl_radius[i])
    
builder.save('CT1.yaml')

Deleted existing CT1.yaml
Saved telescope config to CT1.yaml
